# 0. Чтение данных и базовое ознакомление

In [52]:
# импортируем необходимые библиотеки
import pandas as pd
import numpy as np

In [53]:
# считываем данные
df = pd.read_csv('data/data.csv')

In [54]:
# выясним размерность датасета
print('размерность датасета: ',df.shape)
# проверим корректность загрузки и ознакомимся с полями
df.head()

размерность датасета:  (377185, 18)


,status,private pool,propertyType,street,baths,homeFacts,fireplace,city,schools,sqft,zipcode,beds,state,stories,mls-id,PrivatePool,MlsId,target
0,Active,NaN,Single Family Home,240 Heather Ln,3.5,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",Gas Logs,Southern Pines,"[{'rating': ['4', '4', '7', 'NR', '4', '7', 'N...",2900,28387,4,NC,NaN,NaN,NaN,611019,"$418,000"
1,for sale,NaN,single-family home,12911 E Heroy Ave,3 Baths,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",NaN,Spokane Valley,"[{'rating': ['4/10', 'None/10', '4/10'], 'data...","1,947 sqft",99216,3 Beds,WA,2.0,NaN,NaN,201916904,"$310,000"
2,for sale,NaN,single-family home,2005 Westridge Rd,2 Baths,"{'atAGlanceFacts': [{'factValue': '1961', 'fac...",yes,Los Angeles,"[{'rating': ['8/10', '4/10', '8/10'], 'data': ...","3,000 sqft",90049,3 Beds,CA,1.0,NaN,yes,FR19221027,"$2,895,000"
3,for sale,NaN,single-family home,4311 Livingston Ave,8 Baths,"{'atAGlanceFacts': [{'factValue': '2006', 'fac...",yes,Dallas,"[{'rating': ['9/10', '9/10', '10/10', '9/10'],...","6,457 sqft",75205,5 Beds,TX,3.0,NaN,NaN,14191809,"$2,395,000"
4,for sale,NaN,lot/land,1524 Kiscoe St,NaN,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",NaN,Palm Bay,"[{'rating': ['4/10', '5/10', '5/10'], 'data': ...",NaN,32908,NaN,FL,NaN,NaN,NaN,861745,"$5,000"


In [55]:
# оценим количество пропусков и типы данных
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 377185 entries, 0 to 377184
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   status        337267 non-null  object
 1   private pool  4181 non-null    object
 2   propertyType  342452 non-null  object
 3   street        377183 non-null  object
 4   baths         270847 non-null  object
 5   homeFacts     377185 non-null  object
 6   fireplace     103115 non-null  object
 7   city          377151 non-null  object
 8   schools       377185 non-null  object
 9   sqft          336608 non-null  object
 10  zipcode       377185 non-null  object
 11  beds          285903 non-null  object
 12  state         377185 non-null  object
 13  stories       226470 non-null  object
 14  mls-id        24942 non-null   object
 15  PrivatePool   40311 non-null   object
 16  MlsId         310305 non-null  object
 17  target        374704 non-null  object
dtypes: object(18)
memory usa

In [56]:
# посмотрим на количество явных пропусков с более удобного ракурса
df.isnull().sum()

status           39918
private pool    373004
propertyType     34733
street               2
baths           106338
homeFacts            0
fireplace       274070
city                34
schools              0
sqft             40577
zipcode              0
beds             91282
state                0
stories         150715
mls-id          352243
PrivatePool     336874
MlsId            66880
target            2481
dtype: int64

Описание данных:

- 'status' — статус продажи;
- 'private pool' и 'PrivatePool' — наличие собственного бассейна;
- 'propertyType' — тип объекта недвижимости;
- 'street' — адрес объекта;
- 'baths' — количество ванных комнат;
- 'homeFacts' — сведения о строительстве объекта (содержит несколько типов сведений, влияющих на оценку объекта);
- 'fireplace' — наличие камина;
- 'city' — город;
- 'schools' — сведения о школах в районе;
- 'sqft' — площадь в футах;
- 'zipcode' — почтовый индекс;
- 'beds' — количество спален;
- 'state' — штат;
- 'stories' — количество этажей;
- 'mls-id' и 'MlsId' — идентификатор MLS (Multiple Listing Service, система мультилистинга);
- 'target' — цена объекта недвижимости (целевой признак, который необходимо спрогнозировать).

# 1. Предобработка данных

In [57]:
# для начала удалим записи с пустыми значениями целевой переменной, они явно не участвуют в этом мероприятии
df = df[~df['target'].isna()]
df.shape[0]

In [80]:
# пройдемся по датасету и уберем лишние пробелы и переносы строк
df = df.replace({r'\s+$': '', r'^\s+': ''}, regex=True).replace(r'\n',  ' ', regex=True)

In [64]:
# проанализируем датасет на наличие полных дубликатов
df.duplicated().sum()

50

In [65]:
# удалим полные дубликаты исходного датасета
df = df.drop_duplicates()
df.shape

(377135, 18)

#### Последовательно проверим признаки

In [119]:
columns = df.columns
for column in columns:
    print()
    print(column)
    print()
    print(df[column].value_counts().head(50))
    if df[column].value_counts().shape[0] > 50: 
        print(df[column].value_counts().tail(50))


status

for sale                             156058
Active                               105207
For sale                              43464
foreclosure                            6425
New construction                       5474
Pending                                4702
Pre-foreclosure                        2119
Pre-foreclosure / auction              1560
P                                      1488
Under Contract Show                    1183
/ auction                               936
Under Contract   Showing                793
Active Under Contract                   718
New                                     690
Under Contract                          690
Contingent                              581
Price Change                            563
Auction                                 536
Foreclosed                              459
A Active                                443
for rent                                398
Foreclosure                             343
recently sold          

##### Частный бассейн - 'private pool' и 'PrivatePool'

In [66]:
# проверим пару признаков наличия частного бассейна
# для начала проверим варианты внесения информации
display(df['private pool'].value_counts())
display(df['PrivatePool'].value_counts())

Yes    4181
Name: private pool, dtype: int64

yes    28792
Yes    11518
Name: PrivatePool, dtype: int64

In [67]:
# предположим, что это признаки, созданные в базе данных в разное время
# т.е., информация о наличии бассейна есть либо в одном, либо в другом
# в таком случае, не должно быть записей, в которых одновременно указано наличие в обоих полях
# проверим простым способом - переведем наличие в "1" и сложим оба столбца
# если запись и там, и там, получим в этих записях двойки

# для начала переведем записи в единицы и проверим сохранение количеств значений
df['PrivatePool'] = df['PrivatePool'].apply(lambda x: 1 if x in ['yes', 'Yes'] else 0)
display(df['PrivatePool'].value_counts())
df['private pool'] = df['private pool'].apply(lambda x: 1 if x in ['yes', 'Yes'] else 0)
display(df['private pool'].value_counts())

0    336825
1     40310
Name: PrivatePool, dtype: int64

0    372954
1      4181
Name: private pool, dtype: int64

In [68]:
# поскольку перекодирование столбцов прошло успешно, теперь делаем аггрегирующее поле
# проверка вариантов значений покажет, подтвердилось ли вышеописанное предположение
df['PoolPrivate'] = df['PrivatePool'] + df['private pool']
df['PoolPrivate'].value_counts()

0    332644
1     44491
Name: PoolPrivate, dtype: int64

In [69]:
# схема сработала, изначальные признаки можно удалить
df = df.drop(columns=['private pool', 'PrivatePool'], axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 377135 entries, 0 to 377184
Data columns (total 17 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   status        337218 non-null  object
 1   propertyType  342402 non-null  object
 2   street        377133 non-null  object
 3   baths         270827 non-null  object
 4   homeFacts     377135 non-null  object
 5   fireplace     103112 non-null  object
 6   city          377101 non-null  object
 7   schools       377135 non-null  object
 8   sqft          336585 non-null  object
 9   zipcode       377135 non-null  object
 10  beds          285881 non-null  object
 11  state         377135 non-null  object
 12  stories       226462 non-null  object
 13  mls-id        24942 non-null   object
 14  MlsId         310266 non-null  object
 15  target        374655 non-null  object
 16  PoolPrivate   377135 non-null  int64 
dtypes: int64(1), object(16)
memory usage: 51.8+ MB


##### Идентификаторы MLS - 'mls-id' и 'MlsId'

In [93]:
# проверим пару признаков идентификаторов MLS
# предположение такое - признаки заполнялись в разные периоды времени (как бассейны)
# если это так, значение есть либо в одном поле, либо в другом, проверим
print('Количество записей с одновременно заполненными полями "MlsId" и "mls-id" : ',df[~df['MlsId'].isna() & ~df['mls-id'].isna()].shape[0])

Количество записей с одновременно заполненными полями "MlsId" и "mls-id" :  0


In [94]:
# нет ни одной записи, в которой одновременно были бы непустые значения в обоих полях
# теперь проверим, нет ли одинаковых идентификаторов в обоих полях, и если есть, то идентичные ли записи им соответствуют
# это будет обозначать, что в какой-то момент одно поле для внесения отключили, а второе активировали
# для начала переведем в строчные буквы
df['MlsId'] = df['MlsId'].str.lower()
df['mls-id'] = df['mls-id'].str.lower()
MlsIdList = list(df['MlsId'].unique())
print('уникальных значений MlsId : ',len(MlsIdList))
mls_id_List = list(df['mls-id'].unique())
print('уникальных значений mls-id : ',len(mls_id_List))
print ('при пересечении только nan длина множества должна быть : ', (len(MlsIdList)+len(mls_id_List)-1))
Mls = set(MlsIdList + mls_id_List)
print('уникальных значений кумулятивно по обоим столбцам : ',len(Mls))
if len(Mls) < (len(MlsIdList)+len(mls_id_List)-1):
    print('значения столбцов пересекаются')
else:
    print('значения столбцов не пересекаются')

уникальных значений MlsId :  232929
уникальных значений mls-id :  24907
при пересечении только nan длина множества должна быть :  257835
уникальных значений кумулятивно по обоим столбцам :  248997
значения столбцов пересекаются


In [110]:
q = pd.DataFrame(MlsIdList + mls_id_List)
q.value_counts().head(30)

76595543      2
dcdc449624    2
a10772340     2
14096844      2
dcdc449618    2
43719204      2
77092420      2
14096628      2
9822882       2
77099258      2
34936525      2
14096546      2
o5800906      2
o5800907      2
14096327      2
7710472       2
dcdc449558    2
98189504      2
14095924      2
dcdc449532    2
dcdc449528    2
43691201      2
221373        2
981685        2
dcdc449632    2
98329117      2
34961526      2
1852182       2
1409766       2
98301303      2
dtype: int64

In [72]:
# объединим идентификатор MLS в один столбец и удалим два исходных
df['MLS'] = df['MlsId'].fillna('') + df['mls-id'].fillna('')
#df = df.drop(columns=['MlsId', 'mls-id'], axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 377135 entries, 0 to 377184
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   status        337218 non-null  object
 1   propertyType  342402 non-null  object
 2   street        377133 non-null  object
 3   baths         270827 non-null  object
 4   homeFacts     377135 non-null  object
 5   fireplace     103112 non-null  object
 6   city          377101 non-null  object
 7   schools       377135 non-null  object
 8   sqft          336585 non-null  object
 9   zipcode       377135 non-null  object
 10  beds          285881 non-null  object
 11  state         377135 non-null  object
 12  stories       226462 non-null  object
 13  mls-id        24942 non-null   object
 14  MlsId         310266 non-null  object
 15  target        374655 non-null  object
 16  PoolPrivate   377135 non-null  int64 
 17  MLS           377135 non-null  object
dtypes: int64(1), object(17)


In [73]:
# проверим столбец на распределение количества записей с одинаковыми значениями в поле MLS
MLS_count = pd.DataFrame(df['MLS'].value_counts()).reset_index()
MLS_count['MLS'].value_counts()

1        181459
2         50379
3         15828
4          1265
5            43
6            14
8             3
9             2
7             2
41927         1
40            1
16            1
13            1
12            1
11            1
Name: MLS, dtype: int64

In [74]:
# Просмотрим подробно наиболее часто встречающиеся значения
MLS_count[MLS_count['MLS'] >= 6]

,index,MLS
0,,41927
1,no mls,40
2,no mls #,16
3,"a, houston, tx 77008",13
4,no,12
5,"12a, orlando, fl 32833",11
6,"b, houston, tx 77008",9
7,"1, south boston, ma 02127",9
8,"b, houston, tx 77007",8
9,"2, washington, dc 20002",8


In [111]:
df[df['MLS'] == 'dcdc449528']

,status,propertyType,street,baths,homeFacts,fireplace,city,schools,sqft,zipcode,beds,state,stories,mls-id,MlsId,target,PoolPrivate,MLS,unique
105736,Active,Condo/Townhome/Row Home/Co-Op,1745 N St NW Unit 412,1,"{'atAGlanceFacts': [{'factValue': '2018', 'fac...",NaN,Washington,"[{'rating': ['8', '3', 'NR', '10', '8', '10', ...",815,20036,1,DC,1,NaN,dcdc449528,"$659,900",0,dcdc449528,dcdc4495281745 N St NW Unit 412
126211,New construction,Condo,1745 N St NW UNIT 412,Bathrooms: 1,"{'atAGlanceFacts': [{'factValue': '2018', 'fac...",NaN,Washington,"[{'rating': ['8/10', '3/10'], 'data': {'Distan...",Total interior livable area: 815 sqft,20036,1 bd,DC,1.0,dcdc449528,NaN,"$659,900",0,dcdc449528,dcdc4495281745 N St NW UNIT 412
321310,Active,Victorian,1745 N St NW #412,"1,500","{'atAGlanceFacts': [{'factValue': '2018', 'fac...",NaN,Washington,"[{'rating': ['8', 'NR', '3', 'NR'], 'data': {'...",815,20036,Baths,DC,NaN,NaN,dcdc449528,"659,900",0,dcdc449528,dcdc4495281745 N St NW #412
334890,for sale,condo,1745 N St NW #412,NaN,"{'atAGlanceFacts': [{'factValue': '2017', 'fac...",NaN,Washington,"[{'rating': ['8/10', '3/10'], 'data': {'Distan...",815 sqft,20036,NaN,DC,1.0,NaN,dcdc449528,"$659,900",0,dcdc449528,dcdc4495281745 N St NW #412


In [99]:
df['street'].value_counts().head(10)

Address Not Disclosed    672
Undisclosed Address      517
(undisclosed Address)    391
Address Not Available    175
Unknown Address           72
2103 E State Hwy 21       57
11305 Gulf Fwy            54
17030 Youngblood Rd.      38
NE 58th Cir               34
9470 Lancaster Rd. SW     32
Name: street, dtype: int64

In [100]:
df[df['street'] == '2103 E State Hwy 21']

,status,propertyType,street,baths,homeFacts,fireplace,city,schools,sqft,zipcode,beds,state,stories,mls-id,MlsId,target,PoolPrivate,MLS
14472,Active,Single Family Home,2103 E State Hwy 21,2,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",NaN,Bryan,"[{'rating': ['2', '2', '2', '4', '3', 'NR', 'N...","2,100",77803,4,TX,NaN,NaN,161662954,"$146,000",0,161662954
14978,Active,Single Family Home,2103 E State Hwy 21,2,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",NaN,Bryan,"[{'rating': ['2', '2', '2', '4', '3', 'NR', 'N...","1,740",77803,3,TX,NaN,NaN,1662934,"$126,000",0,1662934
19399,Active,Single Family Home,2103 E State Hwy 21,2,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",NaN,Bryan,"[{'rating': ['2', '2', '2', '4', '3', 'NR', 'N...",901,77803,3,TX,NaN,NaN,1703475,"$43,563",0,1703475
33031,Active,Single Family Home,2103 E State Hwy 21,2,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",NaN,Bryan,"[{'rating': ['2', '2', '2', '4', '3', 'NR', 'N...","1,694",77803,4,TX,NaN,NaN,1662948,"$92,000",0,1662948
34947,Active,Single Family Home,2103 E State Hwy 21,2,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",NaN,Bryan,"[{'rating': ['2', '2', '2', '4', '3', 'NR', 'N...","1,680",77803,3,TX,NaN,NaN,1662940,"$125,000",0,1662940
39641,Active,Single Family Home,2103 E State Hwy 21,2,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",NaN,Bryan,"[{'rating': ['2', '2', '2', '4', '3', 'NR', 'N...","2,108",77803,4,TX,NaN,NaN,1662928,"$112,000",0,1662928
44009,Active,Single Family Home,2103 E State Hwy 21,2,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",NaN,Bryan,"[{'rating': ['2', '2', '2', '4', '3', 'NR', 'N...","1,178",77803,3,TX,NaN,NaN,1750719,"$58,706",0,1750719
45377,Active,Single Family Home,2103 E State Hwy 21,2,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",NaN,Bryan,"[{'rating': ['2', '2', '2', '4', '3', 'NR', 'N...","1,530",77803,3,TX,NaN,NaN,1706044,"$83,893",0,1706044
47214,Active,Single Family Home,2103 E State Hwy 21,2,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",NaN,Bryan,"[{'rating': ['2', '2', '2', '4', '3', 'NR', 'N...","1,530",77803,3,TX,NaN,NaN,1662925,"$91,000",0,1662925
56593,Active,Single Family Home,2103 E State Hwy 21,2,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",NaN,Bryan,"[{'rating': ['2', '2', '2', '4', '3', 'NR', 'N...","2,280",77803,4,TX,NaN,NaN,161662970,"$152,000",0,161662970


In [102]:
# проверим уникальность объекта по сцепке MLS и адреса
df['unique'] = df['MLS'].fillna('') + df['street'].fillna('')

In [108]:
df[~(df['MLS'] == '')]['unique'].value_counts()

87720101709 Leilani Dr              4
457418357222 Tuck St                4
35714428134 Paw Club Dr             4
15124012214 12th Ave W              4
76916002404 N Burr St               4
                                   ..
a444242115332 Taurus Cir            1
blur1831740MLS #: BLUR1831740       1
22907532816 Enfield Dr              1
2190687883302 E 13th St             1
103742333530 N Lake Shore Dr #4B    1
Name: unique, Length: 270364, dtype: int64

In [109]:
df[df['unique'] == '87720101709 Leilani Dr']

,status,propertyType,street,baths,homeFacts,fireplace,city,schools,sqft,zipcode,beds,state,stories,mls-id,MlsId,target,PoolPrivate,MLS,unique
7673,NaN,Single Family,1709 Leilani Dr,3,"{'atAGlanceFacts': [{'factValue': 'No Data', '...",NaN,Houston,"[{'rating': ['7/10', '3/10', '8/10'], 'data': ...","2,165",77055,3,TX,2.0,NaN,8772010,"$552,500",0,8772010,87720101709 Leilani Dr
20370,for sale,single-family home,1709 Leilani Dr,3 Baths,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",yes,Houston,"[{'rating': ['3/10', '8/10', '7/10'], 'data': ...","2,165 sqft",77055,3 Beds,TX,2.0,NaN,8772010,"$552,500",0,8772010,87720101709 Leilani Dr
139865,Active,Traditional,1709 Leilani Dr,NaN,"{'atAGlanceFacts': [{'factValue': '2020', 'fac...",1,Houston,"[{'rating': ['3', '3', '2'], 'data': {'Distanc...","2,165",77055,NaN,TX,2,NaN,8772010,"552,500",0,8772010,87720101709 Leilani Dr
157208,Active,Single Family Home,1709 Leilani Dr,2.5,"{'atAGlanceFacts': [{'factValue': '2020', 'fac...",1,Houston,"[{'rating': ['3', '3', '2', '2', '10', 'NR', '...",2165,77055,3.0,TX,2,NaN,8772010,"$552,500",0,8772010,87720101709 Leilani Dr


In [76]:
# заменим обозначение отсутствия номера MLS на однотипное пустое значение
no_mls = ['no mls', 'no mls #', 'no', 'nomlsid']

In [77]:
# проверим наличие дублей при исключении из датасета того или иного поля
# columns = df.columns
# for column in columns:
#    print(column, ' - ',df.drop(columns=column, axis=1).duplicated().sum())

Обнаруженные проблемы:
- дубли записей при исключении из датасета практически любого из полей
- значения внесены не в те поля (площадь в кровати)
- не числовые значения
- большое количество значений в некоторых столбцах